In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
# Load the trained model
model = load_model('color_detection_model.h5')  

In [ ]:
# Open Camera
video_capture = cv2.VideoCapture(0)  

# ********************************
# Open a video capture object
# video_capture = cv2.VideoCapture('path/to/your/video.mp4')

In [ ]:
color_classes = ['black','blue','brown','green','grey','orange','pink','purple','red','silver','white','yellow']

label_mapping = {idx: color for idx, color in enumerate(color_classes)}

In [ ]:
while True:
    ret, frame = video_capture.read()

    if not ret:
        print('Error opening camera')
        break  # Break the loop if there are no more frames
    # Define regions of interest (ROIs)
    roi1 = frame[50:150, 50:150] 
    roi2 = frame[50:150, 200:300]  
    roi3 = frame[50:150, 350:450]


    # Preprocess and predict on each ROI
    for idx, roi in enumerate([roi1, roi2, roi3]):
        roi_resized = cv2.resize(roi, (64, 64)) / 255.0
        roi_resized = np.reshape(roi_resized, (1, 64, 64, 3))

        prediction = model.predict(roi_resized)
        predicted_label = np.argmax(prediction)

        if predicted_label in label_mapping:
            predicted_color = label_mapping[predicted_label]
        else:
            predicted_color = 'Unknown'

        x1, y1, x2, y2 = 50, 50 + idx * 150, 150, 200 + idx * 150  # Adjust the coordinates based on your ROIs
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.putText(frame, f'Predicted Color: {predicted_color}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    cv2.imshow('Real-Time Color Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()